In [187]:
import re
import nltk
import tinysegmenter
import glob
import json
import pandas as pd
from nltk.collocations import *

In [126]:
def colloc_filter(text):
    '''If return True: collocation is removed
    If return False: collocation is not removed    
    '''
    if re.search("[^ぁ-んァ-ン一-龥ー]+", text):
        return True
    elif len(text)==1 and re.search("[ぁ-んァ-ン]", text):
        return True
    else:
        return False

In [137]:
list_recipe = []

all_files = glob.glob('data/japanese/*')
for file in all_files:
    with open(file,'r',encoding="utf-8") as f:
        contents = f.read()
        list_recipe += contents.split('\n')

list_list_tokens = []

for recipe in list_recipe:
    tokenized = tinysegmenter.tokenize(recipe)
    list_list_tokens.append(tokenized)

tokens = [token for list_tokens in list_list_tokens for token in list_tokens if not colloc_filter(token)]

In [138]:
token_dist = nltk.FreqDist(tokens)

In [146]:
list_keywords = [x[0] for x in token_dist.most_common(1000)]

In [149]:
list_keywords

['簡単',
 '焼き',
 '炒め',
 '味噌',
 '煮',
 '肉',
 'スープ',
 '飯',
 '丼',
 '風',
 '大根',
 '野菜',
 '豚肉',
 '汁',
 '炊き込み',
 'トマト',
 'たっぷり',
 'キャベツ',
 '中華',
 'サラダ',
 '鮭',
 '和え',
 '鶏',
 '菜',
 '味',
 '揚げ',
 '白菜',
 '醤油',
 'はん',
 'カレー',
 'チーズ',
 '豚',
 '豆腐',
 '和風',
 '美味しい',
 '入り',
 '生姜',
 '風味',
 '巻き',
 '寿司',
 '漬け',
 '鶏肉',
 '甘辛',
 '卵',
 'レンジ',
 'じゃ',
 '絶品',
 '家',
 'ねぎ',
 '分',
 'ピーマン',
 '海老',
 'にぎり',
 'さん',
 '茄子',
 '我',
 'ピリ',
 '豚バラ',
 '煮物',
 'バター',
 '草',
 'ツナ',
 'ベーコン',
 'もの',
 '牡蠣',
 'ポタージュ',
 'アボカド',
 'ソース',
 'この',
 'ない',
 'ほうれん',
 'さっぱり',
 '煮込',
 '人参',
 '照り',
 'たけ',
 'イカ',
 'フライパン',
 'ヘルシー',
 '鶏むね',
 'マヨ',
 '玉ねぎ',
 '牛肉',
 '小松菜',
 'ハンバーグ',
 'ブロッコリー',
 'チキン',
 'かぼちゃ',
 '酢',
 '春',
 'なす',
 'だけ',
 'とろ',
 'あんかけ',
 '美味',
 'がい',
 '塩麹',
 'サーモン',
 'チャーハン',
 'ソテー',
 'お弁当',
 '塩',
 '南蛮',
 '鶏も',
 'ごぼう',
 '弁当',
 '本格',
 'ニラ',
 '大葉',
 '豚汁',
 'もやし',
 'あさり',
 '時短',
 'アスパラ',
 'から',
 'しめじ',
 'キムチ',
 '超簡単',
 '炒飯',
 '節約',
 'そぼろ',
 'おつまみ',
 'タコ',
 '甘酢',
 'オクラ',
 '炊飯器',
 '鍋',
 '筍',
 '作る',
 'ぶり',
 '蒸し',
 '唐揚げ',
 '新玉',
 'マリネ',
 '餃子',
 '柚

# Testing

In [253]:
with open('data/test/torikizoku.json','r',encoding = 'utf-8') as f:
    all_elems = json.load(f)

In [254]:
for elem in all_elems:
    tokenized = tinysegmenter.tokenize(elem['text'])
    # score = keyword count
    score = sum(token in list_keywords for token in tokenized)
    elem['score'] = score

In [255]:
# Find most frequently appearing (tag,parents,properties) pair for detected menu items
filtered_elems = [elem for elem in all_elems if elem['score']!=0 and len(elem['text'])<30]

In [256]:
rules = pd.Series({'tag':elem['tag'],'parents':elem['parents'],'properties':elem['properties']} for elem in filtered_elems).value_counts().index[0]

In [257]:
rules

{'tag': 'H5', 'parents': [], 'properties': []}

In [258]:
# Find menu items based on established rules
[elem['text'] for elem in all_elems 
 if elem['tag']==rules['tag'] 
 and elem['parents']==rules['parents'] 
 and elem['properties']==rules['properties']]

['トリキの唐揚',
 'ポテトフライ',
 'とり天',
 '粗挽ポークソーセージ串焼',
 'チキン南蛮',
 'カマンベールコロッケ',
 'ひざなんこつ唐揚',
 'ふんわり山芋の鉄板焼',
 'PDFファイルのダウンロード']

# Experimental

In [100]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
quadgram_measures = nltk.collocations.QuadgramAssocMeasures()

In [101]:
len(list_recipe)

10002

In [102]:
words = [word for recipe in list_recipe for word in recipe]

In [108]:
finder = BigramCollocationFinder.from_words(words)
finder.apply_freq_filter(30)
for colloc in finder.above_score(bigram_measures.pmi,4):
    colloc_text = ''.join(colloc)
    if colloc_filter(colloc_text):
        continue
    else:
        print(colloc_text)

竜田
砂肝
制限
質制
舞茸
定番
糖質
麻婆
料理
時短
人参
小松
圧力
節約
南蛮
冷凍
使用
食べ
赤ワ
基本
薄切
本格
春雨
胡椒
飯器
力鍋
絶品
喜ぶ
弁当
中華
豆腐
添え
油淋
胡麻
炊飯
蓮根
醤油
生姜
新玉
ぞれ
羽先
羽元
玉葱
ネギ
塩麹
漬け
大葉
大根
手羽
そぼ
手軽
黒酢
ナゲ
ご飯
団子
茄子
餃子
柚子
和え
唐揚
ペア
ょう
ゆず
野菜
ゅう
大好
揚げ
我が
子供
ふわ
松菜
ぼろ
ゃぶ
柔ら
甘辛
じゃ
お弁
香る
田揚
白菜
むね
が喜
ャベ
ねぎ
ベツ
だれ
ヘル
込み
っぷ
っぱ
みぞ
供が
が家
味噌
夏野
フィ
やわ
美味
厚揚
ちゃ
春巻
作る
包み
玉ね
つゆ
マヨ
ん草
炒め
ごぼ
詰め
ムネ
煮込
ほう
だけ
角煮
蒸し
和風
さ身
ワイ
アボ
ヒレ
ティ
甘酢
ゲッ
ジュ
噌漬
入り
ゃが
照り
セロ
子胡
り置
らか
わふ
しょ
手作
ぼう
こま
つく
ぼち
さび
ぷり
ミナ
簡単
切り
レモ
ごは
あん
ソテ
サミ
酢あ
味付
たっ
なす
エッ
羽中
煮物
カツ
わら
かず
ぱり
ラダ
飯が
姜焼
ップ
驚き
ケチ
淋鶏
シュ
ュウ
超簡
置き
オイ
作り
巻き
焼き
って
うま
華風
フラ
モモ
スペ
鶏胸
うち
すす
はん
ごま
つま
イエ
鶏む
ひき
ハム
チキ
くね
豚ヒ
さみ
好き
もや
ウマ
旨い
胸肉
ック
ポテ
ゴマ
やし
ない
ジャ
ささ
肉詰
ピリ
ボカ
みそ
たれ
キャ
くだ
挽き
サラ
あげ
ライ
ハニ
冷し
しゃ
バラ
ブロ
かけ
ニラ
菜た
おろ
きゅ
め物
噌炒
から
もも
サク
しい
カレ
うれ
ナッ
リ辛
トマ
香味
ふん
すむ
あっ
れん
ササ
プル
マト
風味
ミコ
イパ
リブ
ウイ
チャ
チョ
げな
ャッ
ハン
うが
モン
肉団
牛肉
くる
辛炒
タミ
ン南
たけ
ルシ
き鳥
ダレ
華炒
スタ
さっ
ら揚
たま
キン
ふっ
ン粉
ーズ
かぼ
エリ
アリ
らあ
塩レ
おつ
ダイ
パリ
しそ
油炒
こん
ニク
パイ
ね肉
ミル
豚バ
ュー
リッ
バタ
がす
豚こ
ーユ
ても
ステ
ッツ
ムチ
ーグ
パプ
ろし
キム
ゲン
梅し
グリ
カド
スパ
テー
まみ
ヨ炒
肉巻
味し
シャ
おか
ピカ
挽肉
鶏団
くら
ロッ
ソー


In [109]:
finder = TrigramCollocationFinder.from_words(words)
finder.apply_freq_filter(30)
for colloc in finder.above_score(trigram_measures.pmi,4):
    colloc_text = ''.join(colloc)
    if colloc_filter(colloc_text):
        continue
    else:
        print(colloc_text)

質制限
糖質制
炊飯器
圧力鍋
竜田揚
小松菜
供が喜
柚子胡
我が家
赤ワイ
子胡椒
お弁当
が喜ぶ
手羽先
そぼろ
手羽元
薄切り
作り置
みぞれ
玉ねぎ
ふわふ
唐揚げ
新玉ね
たっぷ
ナゲッ
れん草
ほうれ
ン南蛮
じゃが
田揚げ
ャベツ
中華風
子供が
油淋鶏
夏野菜
ょうが
ごぼう
生姜焼
柔らか
ぼちゃ
わふわ
厚揚げ
わさび
噌漬け
ヘルシ
やわら
ペアリ
ご飯が
ダイエ
しょう
ィーユ
しゃぶ
手羽中
ボカド
甘酢あ
サラダ
中華炒
スペア
キャベ
ルフィ
煮込み
つくね
ケチャ
さっぱ
すすむ
醤油炒
アボカ
タミナ
ゅうり
野菜た
んつゆ
冷しゃ
っぷり
ささ身
ごはん
かぼち
おかず
味噌漬
くだけ
美味し
ふんわ
酢あん
っぱり
ピリ辛
ササミ
わらか
塩レモ
アリブ
ゲット
きゅう
ャップ
セロリ
がすす
サミコ
鶏むね
イエッ
シュウ
フライ
手作り
照り焼
春巻き
揚げな
肉団子
ふっく
込みハ
むね肉
エット
こって
んにく
あんか
おつま
豚ヒレ
パン粉
ナッツ
甘辛炒
菜たっ
ブロッ
当にも
ら揚げ
うれん
豚こま
ュウマ
り置き
ささみ
ルサミ
鶏そぼ
つまみ
甘辛煮
鶏手羽
肉豆腐
ミルフ
げない
炒め物
ゃがい
超簡単
もやし
肉詰め
鶏団子
味噌炒
ポテト
ムネ肉
ワイン
鶏ムネ
から揚
姜焼き
サクサ
噌炒め
おろし
アスパ
ハンバ
レモン
ライパ
トマト
ビーフ
あっさ
ポン酢
野菜炒
ピカタ
んかけ
辛炒め
肉じゃ
ヒレ肉
フィー
華炒め
オイス
ー醤油
味しい
キン南
焼き鳥
ウマイ
鶏モモ
クサク
クソテ
グリル
豚バラ
油炒め
ニンニ
マヨ炒
ト煮込
チャッ
バーグ
ジュー
うが焼
ロッコ
パイシ
ポーク
チキン
鶏つく
鶏胸肉
っくら
蒸し鶏
んかつ
ヨ炒め
パプリ
レシピ
ベーコ
ューシ
ひき肉
ソテー
かジュ
リック
スタミ
ってり
バルサ
ろとろ
パリパ
チーズ
ツレツ
オーブ
オクラ
レンジ
しめじ
鶏もも
酢炒め
キムチ
イスタ
んわり
バジル
オイル
がいも
挽き肉
タード
マリネ
プリカ
味噌煮
イパン
マト煮
めんつ
ウイン
カツレ
コチュ
らかい
らかさ
スパラ
味噌焼
ソース
らかジ
切り肉
ピーマ
ッコリ
ルキャ
豚しゃ
もも肉
タンド
スパイ
シャキ
インナ
すき焼
豚の角
肉巻き


In [110]:
finder = QuadgramCollocationFinder.from_words(words)
finder.apply_freq_filter(30)
for colloc in finder.above_score(quadgram_measures.pmi,4):
    colloc_text = ''.join(colloc)
    if colloc_filter(colloc_text):
        continue
    else:
        print(colloc_text)

糖質制限
柚子胡椒
供が喜ぶ
竜田揚げ
子供が喜
新玉ねぎ
ふわふわ
うれん草
ペアリブ
冷しゃぶ
ほうれん
赤ワイン
アボカド
フィーユ
ミルフィ
ナゲット
作り置き
かぼちゃ
ダイエッ
たっぷり
菜たっぷ
やわらか
キン南蛮
弁当にも
キャベツ
味噌漬け
しょうが
甘酢あん
がすすむ
ょうが焼
薄切り肉
野菜たっ
ケチャッ
じゃがい
中華炒め
生姜焼き
あんかけ
鶏そぼろ
スペアリ
美味しい
鶏手羽元
醤油炒め
めんつゆ
ふっくら
さっぱり
ルサミコ
揚げない
フライパ
シュウマ
スタミナ
ふんわり
イエット
おつまみ
から揚げ
ブロッコ
きゅうり
ュウマイ
驚きの柔
チャップ
煮込みハ
豚しゃぶ
サクサク
ヘルシー
ウインナ
柔らかさ
甘辛炒め
肉じゃが
バルサミ
こってり
ゃがいも
オイスタ
味噌炒め
ター醤油
照り焼き
塩レモン
ルキャベ
ニンニク
アスパラ
ルフィー
ジューシ
鶏つくね
鶏むね肉
野菜炒め
パプリカ
ト煮込み
込みハン
あっさり
カツレツ
マト煮込
ピリ辛炒
マヨ炒め
チキン南
ワイン煮
柔らか鶏
ロッコリ
豚ヒレ肉
パリパリ
ャーシュ
ガーリッ
ライパン
鶏ムネ肉
スパイシ
トマト煮
おいしい
うが焼き
タルタル
鶏ささみ
豚の角煮
タンドリ
豚の生姜
バター醤
スタード
ヨーグル
リ辛炒め
ンバーグ
シンプル
ハンバー
ベーコン
カリカリ
トマトソ
アレンジ
みハンバ
ポークソ
クリーム
オーブン
キンソテ
ハニーマ
味噌焼き
キンカツ
クソテー
肉の生姜
きフライ
肉と野菜
すき焼き
鶏の唐揚
揚げ簡単
パイシー
豚キムチ
豚バラ巻
ークソテ
肉と玉ね
鶏もも肉
ピーマン
かジュー
ューシー
ーリック
しっとり
トポーク
ジで簡単
ステーキ
チャーシ
きの柔ら
炒め簡単
イスター
ミートロ
チキンナ
ドチキン
鶏ひき肉
ロースト
トローフ
ブン焼き
チキンカ
ッコリー
ールキャ
ンチカツ
ンジで簡
インナー
チキンソ
ター炒め
マスター
ドリーチ
カレー風
豚肉巻き
ロールキ
リーム煮
ンソテー
ストチキ
肉の甘辛
豚バラ肉
ーグルト
チーズ焼
マトソー
ーズ焼き
焼き簡単
ーシュー
スターソ
ンドリー
焼きチキ
ーストポ
肉の味噌
トソース
ソース炒
ストポー
キンステ
トチキン
ーブン焼
スター炒
ルチキン
肉とキャ
り焼きチ
